In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np
import wandb


In [23]:
import gc
import torch
import numpy as np
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import wandb

In [24]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [25]:
class CNN(nn.Module):
    def __init__(
        self,
        input_dimension: tuple,
        number_of_filters: int,
        filter_size: tuple,
        stride: int,
        padding: int,
        max_pooling_size: tuple,
        n_neurons: int,
        n_classes: int,
        conv_activation: nn.Module,
        dense_activation: nn.Module,
        dropout_rate: float,
        use_batchnorm: bool,
        factor: float,
        dropout_organisation: int,
    ):
        super(CNN, self).__init__()

        self.conv_blocks = nn.ModuleList()
        in_channels = input_dimension[0]

        for i in range(5):
            out_channels = int((factor ** i) * number_of_filters)
            out_channels = max(out_channels, 3)

            add_dropout = (i % dropout_organisation) > 0

            conv_block = self.create_conv_block(
                in_channels,
                out_channels,
                filter_size,
                max_pooling_size,
                stride,
                padding,
                conv_activation,
                dropout_rate,
                use_batchnorm,
                add_dropout,
            )
            self.conv_blocks.append(conv_block)
            in_channels = out_channels

        self.flatten = nn.Flatten()

        # Compute the size after conv layers
        dummy_input = torch.ones(1, *input_dimension)
        with torch.no_grad():
            x = dummy_input
            for block in self.conv_blocks:
                x = block(x)
        in_features = x.view(1, -1).shape[1]

        # Define dense layers
        self.dense_block1 = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=n_neurons),
            dense_activation,
            nn.Linear(n_neurons, n_classes),
            nn.LogSoftmax(dim=1),
        )

    def create_conv_block(
        self,
        in_c,
        out_c,
        kernel_size,
        max_pooling_size,
        stride,
        padding,
        conv_activation,
        dropout_rate,
        use_batchnorm,
        add_dropout,
    ):
        layers = [
            nn.Conv2d(in_c, out_c, kernel_size=kernel_size, stride=stride, padding=padding),
            conv_activation
        ]
        if use_batchnorm:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.MaxPool2d(kernel_size=max_pooling_size))
        if add_dropout:
            layers.append(nn.Dropout(p=dropout_rate))
        return nn.Sequential(*layers)

    def forward(self, x):
        for block in self.conv_blocks:
            x = block(x)
        x = self.flatten(x)
        return self.dense_block1(x)


In [26]:
# Test data path
training_data_path = "/kaggle/input/nature-12k/inaturalist_12K/train"


In [27]:
torch.backends.cudnn.benchmark = True

In [28]:
# Fixed configurations
config = {
    'number_of_filters': 64,
    'filter_size': 3,
    'stride': 1,
    'padding': 1,
    'max_pooling_size': 2,
    'n_neurons': 64,
    'n_classes': 10,
    'conv_activation': 'silu',
    'dense_activation': 'sigmoid',
    'dropout_rate': 0.2,
    'use_batchnorm': True,
    'factor': 3,
    'learning_rate': 1e-4,
    'batch_size': 16,
    'epochs': 15,
    'use_augmentation': False,
    'dropout_organisation': 2,
}


In [29]:
wandb.login(key='f15dba29e56f32e9c31d598bce5bc7a3c76de62e')
wandb.init(project="DA6401_Assignment2", config=config, name="fixed_config_train")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [30]:
def get_transform(use_augmentation):
    if use_augmentation:
        return transforms.Compose([
            transforms.RandomCrop(50, padding=1),
            transforms.RandomGrayscale(p=0.1),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(degrees=(0, 20)),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
            )
        ])
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
    ])

In [35]:
def train_model():
    # Load and split dataset
    dataset = ImageFolder(root=training_data_path, transform=get_transform(config['use_augmentation']))
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_set, val_set = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_set, batch_size=config['batch_size'], shuffle=False)

    # Setup activations
    activations = {
        'silu': nn.SiLU(),
        'sigmoid': nn.Sigmoid(),
    }

    # Initialize model
    gc.collect()
    torch.cuda.empty_cache()
    model = CNN(
        input_dimension=(3, 224, 224),
        number_of_filters=config['number_of_filters'],
        filter_size=(config['filter_size'], config['filter_size']),
        stride=config['stride'],
        padding=config['padding'],
        max_pooling_size=(config['max_pooling_size'], config['max_pooling_size']),
        n_neurons=config['n_neurons'],
        n_classes=config['n_classes'],
        conv_activation=activations[config['conv_activation']],
        dense_activation=activations[config['dense_activation']],
        dropout_rate=config['dropout_rate'],
        use_batchnorm=config['use_batchnorm'],
        factor=config['factor'],
        dropout_organisation=config['dropout_organisation'],
    ).to(device)

    # Training setup
    optimizer = Adam(model.parameters(), lr=config['learning_rate'])
    criterion = nn.CrossEntropyLoss()
    
    # Metrics tracking
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    best_val_accuracy = 0.0

    # Training loop
    for epoch in range(config['epochs']):
        # Training phase
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * x.size(0)
            correct += (pred.argmax(1) == y).sum().item()
            total += y.size(0)
            del x, y
            
        train_accuracy = 100 * correct / total
        avg_train_loss = train_loss / total
        train_losses.append(avg_train_loss)
        train_accuracies.append(train_accuracy)

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                pred = model(x)
                loss = criterion(pred, y)
                val_loss += loss.item() * x.size(0)
                correct += (pred.argmax(1) == y).sum().item()
                total += y.size(0)
                del x, y
                
        val_accuracy = 100 * correct / total
        avg_val_loss = val_loss / total
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_accuracy)

        wandb.log({
            'epoch': epoch+1,
            'train_loss': avg_train_loss,
            'train_accuracy': train_accuracy,
            'val_loss': avg_val_loss,
            'val_accuracy': val_accuracy
        })

        print(f"Epoch [{epoch+1}/{config['epochs']}]")
        print(f"Train Loss: {avg_train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%")
        print(f"Val Loss: {avg_val_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%")
        print("-" * 50)

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), "best_model_fixed_config_train.pth")

    # Plot training history
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.title('Loss History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Val Accuracy')
    plt.title('Accuracy History')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

    print(f"Best validation accuracy: {best_val_accuracy:.2f}%")
    print("Training history plot saved as 'training_history.png'")
    
    wandb.finish()
    return model

In [36]:
if __name__ == "__main__":
    model = train_model()

Epoch [1/15]
Train Loss: 2.2338 | Train Accuracy: 18.13%
Val Loss: 2.1703 | Val Accuracy: 21.25%
--------------------------------------------------
Epoch [2/15]
Train Loss: 2.1710 | Train Accuracy: 21.22%
Val Loss: 2.1636 | Val Accuracy: 21.35%
--------------------------------------------------
Epoch [3/15]
Train Loss: 2.1522 | Train Accuracy: 22.38%
Val Loss: 2.1399 | Val Accuracy: 21.80%
--------------------------------------------------
Epoch [4/15]
Train Loss: 2.1276 | Train Accuracy: 23.63%
Val Loss: 2.1052 | Val Accuracy: 23.25%
--------------------------------------------------
Epoch [5/15]
Train Loss: 2.1265 | Train Accuracy: 22.92%
Val Loss: 2.1156 | Val Accuracy: 24.60%
--------------------------------------------------
Epoch [6/15]
Train Loss: 2.1022 | Train Accuracy: 25.24%
Val Loss: 2.0795 | Val Accuracy: 25.95%
--------------------------------------------------
Epoch [7/15]
Train Loss: 2.1012 | Train Accuracy: 24.63%
Val Loss: 2.0914 | Val Accuracy: 24.05%
---------------

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_accuracy,▁▃▄▅▅▆▆▆▆▇▇▇▇██
train_loss,█▆▅▄▄▃▃▃▃▂▂▂▂▁▁
val_accuracy,▁▁▂▃▅▆▄▄▆▆▇▆▅█▇
val_loss,██▇▅▅▄▄▅▃▃▂▃▃▁▂
epoch,15
train_accuracy,27.2159
train_loss,2.04051
val_accuracy,27
val_loss,2.04466
